In [34]:
############################################################################
# Использование API Google SpreadSheets
############################################################################


############################################################################
# Описание API
#
# API для Yandex.Metrica, позволяет программно выполнять действия, 
# например, отправлять файлы.
# Документация по API Yandex.Metrica: 
# https://yandex.ru/dev/metrika/doc/api2/api_v1/attrandmetr/dim_all.html
###########################################################################

In [35]:
# Импортируем библиотеки

import pandas as pd
import datetime
import os
import random
import sys
import requests
import json
from urllib.parse import urlencode
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [36]:
# Базовый URL сервиса

url = 'https://api-metrika.yandex.net/stat/v1/data?'

In [37]:
# Параметры запроса, разделенные '&', в виде 'имя=значение', взятые с сайта Яндекс.Метрики (отчет по "Визитам пользователей")

visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'
url = url + visits

In [38]:
# Код ответа о статусе "200" указывает, что запрос выполнен успешно

vistis_request = requests.get(url)
vistis_request

<Response [200]>

In [39]:
# Получаем файл json

query = requests.get(url)
json_data = json.loads(query.text)

In [40]:
json_data['data']

[{'dimensions': [{'name': '2022-05-04'}], 'metrics': [1312.0]},
 {'dimensions': [{'name': '2022-05-05'}], 'metrics': [1269.0]},
 {'dimensions': [{'name': '2022-05-06'}], 'metrics': [1134.0]},
 {'dimensions': [{'name': '2022-05-03'}], 'metrics': [748.0]},
 {'dimensions': [{'name': '2022-05-02'}], 'metrics': [648.0]},
 {'dimensions': [{'name': '2022-05-07'}], 'metrics': [513.0]},
 {'dimensions': [{'name': '2022-05-08'}], 'metrics': [405.0]}]

In [41]:
# Преобразование полученного json в dataframe

visits_data = pd.DataFrame([(
                             i['dimensions'][0]['name'],
                             i['metrics'][0]) for i in json_data['data']], 
                           columns=['date', 'visits'])
visits_data.head()

,date,visits
0,2022-05-04,1312.0
1,2022-05-05,1269.0
2,2022-05-06,1134.0
3,2022-05-03,748.0
4,2022-05-02,648.0


In [42]:
# Экспортируем преобразованные данные с Яндекс.Метрики в таблицу Google SpreadSheets.
# Укажем адрес электронной почты и путь к нашему файлу с учетными данными

my_mail = 'velocityvectormusic@gmail.com'
path_to_credential = 'testapi-349616-0467ce454ac4.json' 
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [43]:
# Авторизация

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)
gs = gspread.authorize(credentials)

In [44]:
# Создание новой таблицы

spreadsheet_name = 'A new spreadsheet'
sheet = gs.create(spreadsheet_name)

In [45]:
# Настройка прав доступа к новой таблице

sheet.share(my_mail, perm_type='user', role='writer')

In [46]:
# Экспорт датафрэйма в созданную таблицу
# Задаем новое название для внутреннего листа документа

sheet = 'Yandex_visits'
d2g.upload(visits_data, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Yandex_visits' id:1638118855>

In [47]:
# Получение ссылки на созданный документ

sheet = gs.create(spreadsheet_name)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/%s" % sheet.id
spreadsheet_url

'https://docs.google.com/spreadsheets/d/1m09efi6R41s7lAGC8rju80IH0E67oTFLZJzyWC0W5R0'